# Trotterization and Operator Splitting

Recall the form of our Hamiltonian,

$$\hat{H} = \frac{\hat{p}^2}{2m} + V(\hat{x}),$$

and our time evolution unitary,

$$\hat{U} = \exp(-i\hat{H}t/\hbar),$$

which we want to approximate. Now in $\hat{H}$, the kinetic energy term is only dependent on $\hat{p}$ and the potential energy term is only dependent on $\hat{x}$. Naturally, we might consider splitting the Hamiltonian, since if we express our quantum state in terms of the $p$ and then the $x$ basis before doing each transformation, we could get away with only using diagonal operators. But in general,

$$\exp(A+B) \neq \exp(A)\exp(B)$$

if $A$ and $B$ do not commute. However, we can still split the operator if we reduce the error by a method called trotterization, in which we make the approximation

$$e^{A+B} \approx e^{A/2}e^Be^{A/2}. $$

Full details of the error and the motivation for this approximation are given in the Appendix at the bottom of this notebook, but are omitted here so that we can get to the methods as quickly as possible.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import execute, Aer
from qiskit.visualization import plot_histogram
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.circuit.library import QFT

Remember that we'll be switching between the $p$ and $x$ in order to allow $\exp\left(\frac{-i\hat{p}^2}{2m\hbar}\Delta t\right)$ and $\exp\left(\frac{-iV(\hat{x})}{\hbar}\Delta t\right)$ to be diagonalizable. Therefore, we'll perform Fourier transforms in between.

<img src=trotter_creditPeterLove.png>

In [5]:
def second_order_trotter_Hamiltonian(qc, U_p, U_x, qubits):
    '''Second-order Trotterization for the evolution of a state.
    
    Args:
        qc: QuantumCircuit instance
        U_p, U_x: Operators, evolutions for the p and x -basis, respectively
        qubits: indexed QuantumRegister to apply to'''
    num_qubits = len(qubits)
    
    #Diagonalize U_p to calculate sqrt(U_p)
    eigvalues, P = np.linalg.eig(U_p.data)
    D = np.diag(eigvalues)
    sqrtU_p = Operator(P @ np.sqrt(D) @ np.linalg.inv(P))
    
    #Append components to circuit
    qc.append(sqrtU_p, qubits)
    qc.append(QFT(num_qubits, inverse=False), target_qubits)
    qc.append(U_x, qubits)
    qc.append(QFT(num_qubits, inverse=True), target_qubits)
    qc.append(sqrtU_p, qubits)

In general, when we have $H = \sum^m_{k=1} H_k$, the Trotter expansion is

$$ e^{-iHt} \approx \left(\prod^m_{k=1}e^{-iH_{m-k+1}\Delta_t/2} \prod^m_{k=1}e^{-iH_k\Delta_t/2}\right)^N, ~\Delta_t = \frac{t}{N}.$$

In [6]:
def trotter_H_k(qc, H_list, N, t, qubits):
    '''General trotter formula for a Hamiltonian that is a sum of H_list'''
    dt = t/N
    m = len(H_list)
    Op_list = []
    
    #Diagonalize H_k in order to calculate H_k^(dt/2)
    for k in range(m):
        eigvalues, P_k = np.linalg.eig(H_list[k])
        D_k = np.diag(eigvalues)
        Op_list[k] = Operator(P_k @ D_k**(dt/2) @ np.linalg.inv(P))
    
    #Create circuit
    for _ in range(N):
        for k in range(m):
            qc.append(Op_list[k], qubits)
        for k in range(m):
            qc.append(Op_list[m-k+1], qubits)    

Now let us test this on a circuit.

In [ ]:
################# Unfinished ################
# qc = QuantumCircuit(5, 5)

## Appendix: Error of the Approximation
Much of this content was taken from Peter Love's presentation at the Duke summer school.

In general,

$$\exp(A+B) \neq \exp(A)\exp(B)$$

if $A$ and $B$ do not commute. But how do they differ? Let's look at an operator $\hat{U}$:

$$\hat{U} = \exp((A+B)t) = \sum^{\infty}_{n=0}\frac{(A+B)^n}{n!}t^n$$
$$ = 1 + (A+B)t + \frac{(A+B)^2}{2}t^2 + \cdots$$

Compare this to the approximation $\tilde{U}$:

$$ \tilde{U} = \exp(At)\exp(Bt) = \left(\sum^{\infty}_{n=0}\frac{A^n}{n!}t^n\right)\left(\sum^{\infty}_{n=0}\frac{B^n}{n!}t^n\right)$$
$$= \left(1+At+\frac{A^2}{2}t^2+\cdots\right)\left(1+Bt+\frac{B^2}{2}t^2+\cdots\right)$$
$$ = 1 + (A+B)t + \frac{A^2+AB+B^2}{2} + \cdots$$

Looking at the difference between $\tilde{U}$ and $\hat{U}$ up to only the second order terms, we have

$$ \tilde{U} = \hat{U} - \frac{BAt^2}{2} + \frac{ABt^2}{2} + \cdots$$

$$ = \hat{U} + \frac{[A,B]t^2}{2} + \cdots$$

where $[A,B]$ is the called the commutator, and is defined as $[A,B] = AB - BA$.

We could keep doing this to higher order terms; these are described by the [Baker-Campbell-Hausdorff formula](https://en.wikipedia.org/wiki/Baker%E2%80%93Campbell%E2%80%93Hausdorff_formula).
Now how do we get our second-order errors to cancel out? We can try to approximate $e^{A+B}$ with

$$e^{A/2}e^Be^{A/2}.$$

First, the error from the right two terms, $e^C \equiv e^Be^{A/2}$:

$$ C = \log(e^Be^{A/2}) = \frac{A}{2} + B+ \frac{[B,A]}{4} + \frac{[B,[B,A]}{24} + \frac{[A,[A,B]}{48} + \cdots$$

Then we include the $e^{A/2}$ term that was on the left:

$$ \log(e^{A/2}e^C) = \frac{A}{2} + C + \frac{[A,C]}{4} + \frac{[A,[A,C]}{48} + \frac{[C,[C,A]}{24} + \cdots $$

$$ = A+B + \frac{[B,A]}{4}+\frac{[A,B]}{4} + \cdots$$

$$ = A + B - \frac{[B,[A,B]]}{12} - \frac{[A,[A,B]]}{24}+ \cdots, $$

where in the last step we used the property that $[A,B] = -[B,A]$. And we have successfully eliminated our 2nd-order error terms.